## Phase 2 – Data Cleaning Summary

- Removed duplicate records across all datasets
- Standardized date formats and categorical values
- Applied business rules to clean invalid values
- Created derived KPIs such as revenue and warranty completion flag
- Saved cleaned, analysis-ready datasets for ETL loading


In [16]:
sales.to_csv("../data/cleaned/sales_cleaned.csv", index=False)
products.to_csv("../data/cleaned/products_cleaned.csv", index=False)
categories.to_csv("../data/cleaned/categories_cleaned.csv", index=False)
stores.to_csv("../data/cleaned/stores_cleaned.csv", index=False)
warranty.to_csv("../data/cleaned/warranty_cleaned.csv", index=False)


In [15]:
# Validate sales ↔ products
invalid_products = sales[~sales["product_id"].isin(products["Product_ID"])]

# Validate sales ↔ stores
invalid_stores = sales[~sales["store_id"].isin(stores["Store_ID"])]

print("Invalid products after cleaning:", invalid_products.shape[0])
print("Invalid stores after cleaning:", invalid_stores.shape[0])


Invalid products after cleaning: 0
Invalid stores after cleaning: 0


In [14]:
warranty["year"] = warranty["claim_date"].dt.year
warranty["month"] = warranty["claim_date"].dt.month


In [13]:
warranty["is_completed"] = np.where(
    warranty["repair_status"] == "Completed", 1, 0
)


In [12]:
sales["revenue"] = sales["quantity"] * sales.merge(
    products[["Product_ID", "Price"]],
    left_on="product_id",
    right_on="Product_ID",
    how="left"
)["Price"]


In [11]:
# Fix date format
warranty["claim_date"] = pd.to_datetime(warranty["claim_date"], errors="coerce")

# Standardize repair status
warranty["repair_status"] = warranty["repair_status"].str.strip().str.title()

# Remove duplicates
warranty = warranty.drop_duplicates()

warranty.head()


,claim_id,claim_date,sale_id,repair_status
0,CL-58750,2024-01-30,YG-8782,Completed
1,CL-8874,2024-06-25,QX-999001,Pending
2,CL-14486,2024-08-13,JG-46890,Pending
3,CL-42187,2024-09-19,XJ-1731,Pending
4,CL-37590,2024-09-16,FG-95080,Completed


In [10]:
stores = stores.drop_duplicates()
stores.head()


,Store_ID,Store_Name,City,Country
0,ST-1,Apple Fifth Avenue,New York,United States
1,ST-2,Apple Union Square,San Francisco,United States
2,ST-3,Apple Michigan Avenue,Chicago,United States
3,ST-4,Apple The Grove,Los Angeles,United States
4,ST-5,Apple SoHo,New York,United States


In [9]:
categories = categories.drop_duplicates()
categories.head()


,category_id,category_name
0,CAT-1,Laptop
1,CAT-2,Audio
2,CAT-3,Tablet
3,CAT-4,Smartphone
4,CAT-5,Wearable


In [8]:
# Fix launch date format
products["Launch_Date"] = pd.to_datetime(products["Launch_Date"], errors="coerce")

# Remove products with invalid price
products = products[products["Price"] > 0]

products.head()


,Product_ID,Product_Name,Category_ID,Launch_Date,Price
0,P-1,MacBook,CAT-1,2023-09-17,1149
1,P-2,MacBook Air (M1),CAT-1,2023-11-11,1783
2,P-3,MacBook Air (M2),CAT-1,2020-05-24,1588
3,P-4,MacBook Pro 13-inch,CAT-1,2021-01-17,1351
4,P-5,MacBook Pro 14-inch,CAT-1,2024-05-12,768


In [7]:
sales.head()


,sale_id,sale_date,store_id,product_id,quantity,year,month
0,YG-8782,2023-06-16,ST-10,P-38,10,2023,6
1,QX-999001,2022-04-13,ST-63,P-48,10,2022,4
2,JG-46890,2021-07-05,ST-26,P-79,5,2021,7
3,XJ-1731,2022-07-20,ST-15,P-24,9,2022,7
4,FG-95080,2022-03-18,ST-35,P-69,7,2022,3


In [6]:
sales["year"] = sales["sale_date"].dt.year
sales["month"] = sales["sale_date"].dt.month


In [5]:
sales = sales[sales["quantity"] > 0]


In [4]:
sales["sale_date"] = pd.to_datetime(sales["sale_date"], errors="coerce")


C:\Users\Admin\AppData\Local\Temp\ipykernel_13960\2140116193.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  sales["sale_date"] = pd.to_datetime(sales["sale_date"], errors="coerce")


In [3]:
sales_before = sales.shape[0]
sales = sales.drop_duplicates()
sales_after = sales.shape[0]

print("Duplicates removed:", sales_before - sales_after)


Duplicates removed: 0


In [2]:
sales = pd.read_csv("../data/raw/sales.csv")
products = pd.read_csv("../data/raw/products.csv")
categories = pd.read_csv("../data/raw/category.csv")
stores = pd.read_csv("../data/raw/stores.csv")
warranty = pd.read_csv("../data/raw/warranty.csv")


In [1]:
import pandas as pd
import numpy as np

# Phase 2: Data Cleaning & Preparation

Objective:
Clean raw retail datasets by handling missing values, fixing data types,
removing duplicates, and creating derived columns for analysis.
